In [329]:
import ast
import importlib
import re
import json

In [330]:
# Response from LLM
res = '''# This is a sample np.array(np.ones(5)) and another np.sum example
# Comment to check inline many functions

import numpy as np
from matplotlib.pyplot import show
 
x = np.linspace(0, 1, 100)
y = np.sin(2 * np.pi * x)

show()
pyplot.plot(x, y)
pyplot.show()
'''

# Extract imports from the file

In [331]:
packages = []
import_tree = ast.parse(res)

for node in ast.walk(import_tree):
    if 'names' in node.__dir__():
        # import numpy.array as npArray
        if 'module' in node.__dir__():
            # from matplotlib.pyplot import show
            packages.append({"module": node.module, "name": node.names[0].name})
        else:
            packages.append({"name": node.names[0].name, "asname": node.names[0].asname})

print(packages)

[{'name': 'numpy', 'asname': 'np'}, {'module': 'matplotlib.pyplot', 'name': 'show'}]


# Replace alias with package name

In [264]:
lines = res.splitlines()

In [265]:
nps = re.findall(fr'{packages[0]["asname"]}\.[a-zA-Z]+', res)

In [266]:
print(nps)

['np.array', 'np.ones', 'np.sum', 'np.linspace', 'np.sin', 'np.pi']


In [267]:
pakNames = []

In [279]:
'module' in packages[-1]

True

In [347]:
documentations = []

In [348]:
for package in packages:
    if "module" in package:
        package_module = package["module"]
        module_function = package["name"]
        
        spec = importlib.util.find_spec(package_module)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        function_definition = getattr(module, module_function)
        documentation = pydoc.render_doc(function_definition)
        documentations.append({"module": ".".join([package_module, module_function]), "documentation": documentation})
    else:
        last_module = None
        package_module = package["name"]
        module_function = package["asname"]

        if module_function is None:
            package_references = re.findall(rf"{package['name']}\.[a-zA-Z]+", res)
            package_references = [package_reference.rsplit('.', 1) for package_reference in package_references]
            
            for package_module, module_function in package_references:
                spec = importlib.util.find_spec(package_module)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
                function_definition = getattr(module, module_function)
                documentation = pydoc.render_doc(function_definition)
                documentaions.append({"module": ".".join([package_module, module_function]), "documentation": documentation})
        else:
            package_references = re.findall(rf"{package['asname']}\.[a-zA-Z]+", res)
            package_references = [re.sub(module_function, package_module, package_reference) for package_reference in package_references]    
            for package_reference in package_references:
                package_module, package_function = package_reference.split(".")
                if last_module != package_module:
                    spec = importlib.util.find_spec(package_module)
                    module = importlib.util.module_from_spec(spec)
                    spec.loader.exec_module(module)
                    last_module = module
                function_definition = getattr(module, package_function)
                documentation = pydoc.render_doc(function_definition)
                documentations.append({"module": package_reference, "documentation": documentation})

with open("example-documentation.json", "w") as json_file:
    json.dump(documentations, json_file)

/tmp/ipykernel_63266/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_63266/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_63266/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_63266/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_63266/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some

In [349]:
re.findall(r'np\.[a-zA-Z]+', res)

['np.array', 'np.ones', 'np.sum', 'np.linspace', 'np.sin', 'np.pi']

In [284]:
print(documentations)

[{'module': 'matplotlib.pyplot.show', 'documentation': "Python Library Documentation: function show in module matplotlib.pyplot\n\ns\x08sh\x08ho\x08ow\x08w(*args, **kwargs) -> 'None'\n    Display all open figures.\n    \n    Parameters\n    ----------\n    block : bool, optional\n        Whether to wait for all figures to be closed before returning.\n    \n        If `True` block and run the GUI main loop until all figure windows\n        are closed.\n    \n        If `False` ensure that all figure windows are displayed and return\n        immediately.  In this case, you are responsible for ensuring\n        that the event loop is running to have responsive figures.\n    \n        Defaults to True in non-interactive mode and to False in interactive\n        mode (see `.pyplot.isinteractive`).\n    \n    See Also\n    --------\n    ion : Enable interactive mode, which shows / updates the figure after\n          every plotting command, so that calling ``show()`` is not necessary.\n    io

In [232]:
pakDef = [re.sub(packages[0]["asname"], packages[0]["name"], np) for np in nps]

In [233]:
pakDef

['numpy.array',
 'numpy.ones',
 'numpy.sum',
 'numpy.linspace',
 'numpy.sin',
 'numpy.pi']

In [214]:
module = importlib.import_module("matplotlib.pyplot")

In [218]:
module = importlib.import_module("matplotlib")

In [234]:
help(importlib.util.find_spec)

Help on function find_spec in module importlib.util:

find_spec(name, package=None)
    Return the spec for the specified module.
    
    First, sys.modules is checked to see if the module was already imported. If
    so, then sys.modules[name].__spec__ is returned. If that happens to be
    set to None, then ValueError is raised. If the module is not in
    sys.modules, then sys.meta_path is searched for a suitable spec with the
    value of 'path' given to the finders. None is returned if no spec could
    be found.
    
    If the name is for submodule (contains a dot), the parent module is
    automatically imported.
    
    The name and package arguments work the same as importlib.import_module().
    In other words, relative module names (with leading dots) work.



In [235]:
help(importlib.util.module_from_spec)

Help on function module_from_spec in module importlib._bootstrap:

module_from_spec(spec)
    Create a module based on the provided spec.



In [239]:
spec = importlib.util.find_spec("numpy")
print(help(spec.loader.exec_module))

Help on method exec_module in module importlib._bootstrap_external:

exec_module(module) method of _frozen_importlib_external.SourceFileLoader instance
    Execute the module.

None


In [301]:
spec = importlib.util.find_spec("matplotlib")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [327]:
import matplotlib

In [328]:
matplotlib.pyplot.show

<function matplotlib.pyplot.show(*args, **kwargs) -> 'None'>

In [311]:
func = eval("matplotlib"+".pyplot")

In [315]:
getattr(module, "pyplot.show")

AttributeError: module 'matplotlib' has no attribute 'pyplot.show'

In [314]:
getattr(getattr(module, "pyplot.show"), "")

AttributeError: module 'matplotlib' has no attribute 'show'

In [ ]:
for function in ["pyplot", "show"]:
    func = getattr(

In [307]:
import matplotlib

In [309]:
matplotlib.pyplot.__doc__

'\n`matplotlib.pyplot` is a state-based interface to matplotlib. It provides\nan implicit,  MATLAB-like, way of plotting.  It also opens figures on your\nscreen, and acts as the figure GUI manager.\n\npyplot is mainly intended for interactive plots and simple cases of\nprogrammatic plot generation::\n\n    import numpy as np\n    import matplotlib.pyplot as plt\n\n    x = np.arange(0, 5, 0.1)\n    y = np.sin(x)\n    plt.plot(x, y)\n\nThe explicit object-oriented API is recommended for complex plots, though\npyplot is still usually used to create the figure and often the axes in the\nfigure. See `.pyplot.figure`, `.pyplot.subplots`, and\n`.pyplot.subplot_mosaic` to create figures, and\n:doc:`Axes API </api/axes_api>` for the plotting methods on an Axes::\n\n    import numpy as np\n    import matplotlib.pyplot as plt\n\n    x = np.arange(0, 5, 0.1)\n    y = np.sin(x)\n    fig, ax = plt.subplots()\n    ax.plot(x, y)\n\n\nSee :ref:`api_interfaces` for an explanation of the tradeoffs betwee

In [297]:
import importlib.util
import inspect

def get_function_docs(module_name, function_name):
    try:
        # Import the module dynamically
        spec = importlib.util.find_spec(module_name)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        functions = function_name.split(".")

        getattr(getattr(module, functions[0]), function[1])

        func = getattr(module, functions[0])
        for function in functions[1:]:
            func = getattr(func, function)
        print(func)
        # Get the function dynamically
        function = getattr(module, function_name)

        # Use inspect to get documentation
        docstring = inspect.getdoc(func)
        return docstring
    except (ImportError, AttributeError):
        return None

if __name__ == "__main__":
    module_name = "matplotlib"
    function_name = "pyplot.show"

    docstring = get_function_docs(module_name, function_name)

    if docstring:
        print(f"Documentation for {module_name}.{function_name}:\n{docstring}\n")
    else:
        print(f"No documentation found for {module_name}.{function_name}\n")

No documentation found for matplotlib.pyplot.show



In [292]:
import matplotlib

In [294]:
matplotlib.pyplot.show.__doc__

"\n    Display all open figures.\n\n    Parameters\n    ----------\n    block : bool, optional\n        Whether to wait for all figures to be closed before returning.\n\n        If `True` block and run the GUI main loop until all figure windows\n        are closed.\n\n        If `False` ensure that all figure windows are displayed and return\n        immediately.  In this case, you are responsible for ensuring\n        that the event loop is running to have responsive figures.\n\n        Defaults to True in non-interactive mode and to False in interactive\n        mode (see `.pyplot.isinteractive`).\n\n    See Also\n    --------\n    ion : Enable interactive mode, which shows / updates the figure after\n          every plotting command, so that calling ``show()`` is not necessary.\n    ioff : Disable interactive mode.\n    savefig : Save the figure to an image file instead of showing it on screen.\n\n    Notes\n    -----\n    **Saving figures to file and showing a window at the same ti

In [220]:
help(getattr(getattr(module, "pyplot"), "show"))

Help on function show in module matplotlib.pyplot:

show(*args, **kwargs) -> 'None'
    Display all open figures.
    
    Parameters
    ----------
    block : bool, optional
        Whether to wait for all figures to be closed before returning.
    
        If `True` block and run the GUI main loop until all figure windows
        are closed.
    
        If `False` ensure that all figure windows are displayed and return
        immediately.  In this case, you are responsible for ensuring
        that the event loop is running to have responsive figures.
    
        Defaults to True in non-interactive mode and to False in interactive
        mode (see `.pyplot.isinteractive`).
    
    See Also
    --------
    ion : Enable interactive mode, which shows / updates the figure after
          every plotting command, so that calling ``show()`` is not necessary.
    ioff : Disable interactive mode.
    savefig : Save the figure to an image file instead of showing it on screen.
    
    N

In [215]:
print(module.__doc__)


`matplotlib.pyplot` is a state-based interface to matplotlib. It provides
an implicit,  MATLAB-like, way of plotting.  It also opens figures on your
screen, and acts as the figure GUI manager.

pyplot is mainly intended for interactive plots and simple cases of
programmatic plot generation::

    import numpy as np
    import matplotlib.pyplot as plt

    x = np.arange(0, 5, 0.1)
    y = np.sin(x)
    plt.plot(x, y)

The explicit object-oriented API is recommended for complex plots, though
pyplot is still usually used to create the figure and often the axes in the
figure. See `.pyplot.figure`, `.pyplot.subplots`, and
`.pyplot.subplot_mosaic` to create figures, and
:doc:`Axes API </api/axes_api>` for the plotting methods on an Axes::

    import numpy as np
    import matplotlib.pyplot as plt

    x = np.arange(0, 5, 0.1)
    y = np.sin(x)
    fig, ax = plt.subplots()
    ax.plot(x, y)


See :ref:`api_interfaces` for an explanation of the tradeoffs between the
implicit and explicit int

In [203]:
module = importlib.import_module(packages[0]["name"])

In [196]:
del numpy

In [210]:
print(help(getattr(module, pakDef[0].split(".")[-1])))

Help on built-in function array in module numpy:

array(...)
    array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
          like=None)
    
    Create an array.
    
    Parameters
    ----------
    object : array_like
        An array, any object exposing the array interface, an object whose
        ``__array__`` method returns an array, or any (nested) sequence.
        If object is a scalar, a 0-dimensional array containing object is
        returned.
    dtype : data-type, optional
        The desired data-type for the array. If not given, NumPy will try to use
        a default ``dtype`` that can represent the values (by applying promotion
        rules when necessary.)
    copy : bool, optional
        If true (default), then the object is copied.  Otherwise, a copy will
        only be made if ``__array__`` returns a copy, if obj is a nested
        sequence, or if a copy is needed to satisfy any of the other
        requirements (``dtype``, ``order``, e

In [170]:
re.sub(fr'{packages[0]["asname"]}', packages[0]["name"], nps)

TypeError: expected string or bytes-like object, got 'list'

In [153]:
print(pydoc.render_doc(re.search))

Python Library Documentation: function search in module re

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a Match object, or None if no match was found.



In [152]:
print(pydoc.render_doc(re.match))

Python Library Documentation: function match in module re

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a Match object, or None if no match was found.



In [143]:
re.sub(fr'{packages[0]["asname"]}\.[a-zA-Z]+', packages[0]['name'], res)

'# This is a sample numpy(numpy(5)) and another numpy example\n# Comment to check inline many functions\n\nimport numpy as np\nimport numpy.array as npArray\nfrom matplotlib.pyplot import show\n \nx = numpy(0, 1, 100)\ny = numpy(2 * numpy * x)\n\npyplot.plot(x, y)\npyplot.show()\n'

In [116]:
print(help(re.sub))

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.

None


In [113]:
print(re.__doc__)

Support for regular expressions (RE).

This module provides regular expression matching operations similar to
those found in Perl.  It supports both 8-bit and Unicode strings; both
the pattern and the strings being processed can contain null bytes and
characters outside the US ASCII range.

Regular expressions can contain both special and ordinary characters.
Most ordinary characters, like "A", "a", or "0", are the simplest
regular expressions; they simply match themselves.  You can
concatenate ordinary characters, so last matches the string 'last'.

The special characters are:
    "."      Matches any character except a newline.
    "^"      Matches the start of the string.
    "$"      Matches the end of the string or just before the newline at
             the end of the string.
    "*"      Matches 0 or more (greedy) repetitions of the preceding RE.
             Greedy means that it will match as many repetitions as possible.
    "+"      Matches 1 or more (greedy) repetitions of t

In [115]:
print(help(re))

Help on package re:

NAME
    re - Support for regular expressions (RE).

MODULE REFERENCE
    https://docs.python.org/3.11/library/re.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module provides regular expression matching operations similar to
    those found in Perl.  It supports both 8-bit and Unicode strings; both
    the pattern and the strings being processed can contain null bytes and
    characters outside the US ASCII range.
    
    Regular expressions can contain both special and ordinary characters.
    Most ordinary characters, like "A", "a", or "0", are the simplest
    regular expressions; they simply match themselves.  You can
    concatenate ordinary characters, so l

In [109]:
print(pydoc.render_doc(re.sub))

Python Library Documentation: function sub in module re

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.



In [108]:
print(pydoc.render_doc(re.search))

Python Library Documentation: function search in module re

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a Match object, or None if no match was found.



# The problem is that there might be other definitionas similar to this that can not be detected

In [106]:
print(re.Pattern.__doc__)
# BTW, This has huge doc for other two methods

Compiled regular expression object.


In [102]:
print(re.sub.__doc__)

Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.


In [103]:
print(help(re.sub))

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.

None


In [104]:
print(pydoc.render_doc(re.sub))

Python Library Documentation: function sub in module re

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.

